In [2]:
# CAFA-6 Protein Function Prediction - Baseline Model
# This notebook trains a simple linear classifier on ESM2 embeddings

import numpy as np
import pandas as pd
import os
import scipy.sparse as sp
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

print("✅ All imports loaded")

✅ All imports loaded


In [3]:
import os
def get_data_path(filename):
    """
    Returns the correct path whether running locally or on Kaggle.
    
    Args:
        filename: Like 'Train/train_terms.tsv' or 'Test/testsuperset.fasta'
    
    Returns:
        Full path to the file
    """
    # Check if we're on Kaggle
    if os.path.exists('/kaggle/input'):
        # On Kaggle, files are in /kaggle/input/cafa-6-protein-function-prediction/
        base_path = '/kaggle/input/cafa-6-protein-function-prediction'
    else:
        # Locally, files are in current directory
        base_path = '.'
    
    return os.path.join(base_path, filename)

In [4]:
# Step 1: Load ESM2 Embeddings
print("=== Loading ESM2 Embeddings ===")

def find_embedding_dir(possible_names):
    """Find embedding directory (works locally and on Kaggle)"""
    if os.path.exists('/kaggle/input'):
        for name in possible_names:
            kaggle_path = f'/kaggle/input/{name}'
            if os.path.exists(kaggle_path):
                return kaggle_path
    else:
        for name in possible_names:
            if os.path.exists(name):
                return name
    return None

# Try multiple possible names (local and Kaggle)
esm2_possible_names = [
    'cafa-5-ems-2-embeddings-numpy',  # Kaggle dataset name
    'EMS2_Embeddings',                 # Local name option 1
    'ems2-embeddings'                   # Local name option 2
]

esm2_dir = find_embedding_dir(esm2_possible_names)

if esm2_dir:
    print(f"📁 Found ESM2 embeddings at: {esm2_dir}")
    esm2_train_emb = np.load(f'{esm2_dir}/train_embeddings.npy')
    esm2_train_ids = np.load(f'{esm2_dir}/train_ids.npy')
    esm2_test_emb = np.load(f'{esm2_dir}/test_embeddings.npy')
    esm2_test_ids = np.load(f'{esm2_dir}/test_ids.npy')
    
    print(f"✅ ESM2 Train: {esm2_train_emb.shape} ({len(esm2_train_ids)} proteins)")
    print(f"✅ ESM2 Test: {esm2_test_emb.shape} ({len(esm2_test_ids)} proteins)")
    print(f"   Embedding dimension: {esm2_train_emb.shape[1]}")
else:
    raise ValueError(f"❌ ESM2 embeddings not found! Tried: {esm2_possible_names}")

=== Loading ESM2 Embeddings ===
📁 Found ESM2 embeddings at: EMS2_Embeddings
✅ ESM2 Train: (142246, 1280) (142246 proteins)
✅ ESM2 Test: (141864, 1280) (141864 proteins)
   Embedding dimension: 1280


In [5]:
# Step 2: Load training labels and find common proteins
print("=== Loading Training Labels ===")

train_terms = pd.read_csv(get_data_path('Train/train_terms.tsv'), sep='\t')
print(f"Training labels: {len(train_terms)} protein-GO term pairs")
print(f"Unique proteins: {train_terms['EntryID'].nunique()}")
print(f"Unique GO terms: {train_terms['term'].nunique()}")

# Find proteins that are in both embeddings and labels
train_protein_set = set(train_terms['EntryID'].unique())
embedding_protein_set = set(esm2_train_ids)
overlap = train_protein_set & embedding_protein_set

print(f"\nProteins in both: {len(overlap)}")
print(f"Proteins only in labels: {len(train_protein_set - embedding_protein_set)}")
print(f"Proteins only in embeddings: {len(embedding_protein_set - train_protein_set)}")

=== Loading Training Labels ===
Training labels: 537027 protein-GO term pairs
Unique proteins: 82404
Unique GO terms: 26125

Proteins in both: 79268
Proteins only in labels: 3136
Proteins only in embeddings: 62978


In [6]:
# Step 3: Create target matrix
print("=== Creating Target Matrix ===")

common_proteins = list(overlap)
print(f"Using {len(common_proteins)} proteins that have both embeddings and labels")

# Filter embeddings to common proteins
esm2_id_to_idx = {pid: idx for idx, pid in enumerate(esm2_train_ids)}
common_indices = [esm2_id_to_idx[pid] for pid in common_proteins if pid in esm2_id_to_idx]
esm2_train_emb_filtered = esm2_train_emb[common_indices]

# Filter training labels and create target matrix
train_terms_filtered = train_terms[train_terms['EntryID'].isin(common_proteins)].copy()
all_go_terms = sorted(train_terms_filtered['term'].unique())
print(f"Total GO terms to predict: {len(all_go_terms)}")

# Create target matrix using pandas pivot
protein_to_idx = {pid: idx for idx, pid in enumerate(common_proteins)}
train_terms_filtered['protein_idx'] = train_terms_filtered['EntryID'].map(protein_to_idx)

target_df = train_terms_filtered.pivot_table(
    index='protein_idx',
    columns='term',
    values='EntryID',
    aggfunc='count',
    fill_value=0
)

target_matrix = sp.csr_matrix(target_df.values.astype(np.float32))
all_go_terms = target_df.columns.tolist()

print(f"✅ Target matrix: {target_matrix.shape}")
print(f"   Density: {target_matrix.nnz / (target_matrix.shape[0] * target_matrix.shape[1]):.4%}")
print(f"   Avg GO terms per protein: {target_matrix.sum(axis=1).mean():.2f}")

=== Creating Target Matrix ===
Using 79268 proteins that have both embeddings and labels
Total GO terms to predict: 25980
✅ Target matrix: (79268, 25980)
   Density: 0.0256%
   Avg GO terms per protein: 6.66


In [7]:
# Step 4: Split data into train/validation
print("=== Splitting Data ===")

train_idx, val_idx = train_test_split(
    np.arange(len(common_proteins)),
    test_size=0.2,
    random_state=42
)

X_train = esm2_train_emb_filtered[train_idx]
X_val = esm2_train_emb_filtered[val_idx]
y_train = target_matrix[train_idx]
y_val = target_matrix[val_idx]

print(f"Train: {X_train.shape[0]} proteins")
print(f"Validation: {X_val.shape[0]} proteins")

=== Splitting Data ===
Train: 63414 proteins
Validation: 15854 proteins


In [8]:
# Step 5: Build PyTorch Model
print("=== Building Model ===")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class ProteinClassifier(nn.Module):
    """Simple linear classifier: embedding -> GO term logits"""
    def __init__(self, input_dim, output_dim):
        super(ProteinClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.fc(x)

model = ProteinClassifier(X_train.shape[1], y_train.shape[1]).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Model: {X_train.shape[1]} -> {y_train.shape[1]}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

=== Building Model ===
Using device: cpu
Model: 1280 -> 25980
Parameters: 33,280,380


In [9]:
# Step 6: Create DataLoader (Memory-optimized)
print("=== Creating DataLoader ===")

class ProteinDataset(Dataset):
    """Dataset for protein embeddings and GO term labels (memory efficient)"""
    def __init__(self, embeddings, targets):
        # Keep embeddings as numpy, convert to tensor on-the-fly
        self.embeddings = embeddings
        # Keep targets as sparse matrix - convert to dense only for batches
        self.targets = targets
    
    def __len__(self):
        return len(self.embeddings)
    
    def __getitem__(self, idx):
        # Convert only this one row to dense (much more memory efficient)
        emb = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        # Get single row from sparse matrix and convert to dense
        target = torch.tensor(self.targets[idx].toarray().flatten(), dtype=torch.float32)
        return emb, target

train_dataset = ProteinDataset(X_train, y_train)
val_dataset = ProteinDataset(X_val, y_val)

# Reduce batch size to save memory
batch_size = 32  # Reduced from 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")
print(f"Batch size: {batch_size} (reduced for memory efficiency)")

=== Creating DataLoader ===
Train batches: 1982, Val batches: 496
Batch size: 32 (reduced for memory efficiency)


In [10]:
# Step 7: Training Loop (Memory-optimized)
print("=== Training Model ===")

import gc  # For garbage collection

num_epochs = 3
print(f"Training for {num_epochs} epochs...")

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for batch_idx, (emb_batch, target_batch) in enumerate(train_loader):
        emb_batch = emb_batch.to(device)
        target_batch = target_batch.to(device)
        
        outputs = model(emb_batch)
        loss = criterion(outputs, target_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_value = loss.item()
        train_loss += loss_value
        
        # Print progress and clean up memory
        if (batch_idx + 1) % 200 == 0:
            print(f"  Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss_value:.4f}")
            # Clean up memory after printing
            del outputs, loss
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            gc.collect()
        elif (batch_idx + 1) % 100 == 0:
            # Clean up memory periodically (but not on print batches)
            del outputs, loss
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            gc.collect()
    
    avg_train_loss = train_loss / len(train_loader)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for emb_batch, target_batch in val_loader:
            emb_batch = emb_batch.to(device)
            target_batch = target_batch.to(device)
            outputs = model(emb_batch)
            val_loss += criterion(outputs, target_batch).item()
            del outputs  # Free memory
    
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
    # Clean up after each epoch
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("✅ Training complete!")

=== Training Model ===
Training for 3 epochs...
  Epoch 1/3, Batch 200/1982, Loss: 0.0164
  Epoch 1/3, Batch 400/1982, Loss: 0.0092
  Epoch 1/3, Batch 600/1982, Loss: 0.0048
  Epoch 1/3, Batch 800/1982, Loss: 0.0033
  Epoch 1/3, Batch 1000/1982, Loss: 0.0036
  Epoch 1/3, Batch 1200/1982, Loss: 0.0030
  Epoch 1/3, Batch 1400/1982, Loss: 0.0032
  Epoch 1/3, Batch 1600/1982, Loss: 0.0031
  Epoch 1/3, Batch 1800/1982, Loss: 0.0027
Epoch 1/3 - Train Loss: 0.0151, Val Loss: 0.0022
  Epoch 2/3, Batch 200/1982, Loss: 0.0018
  Epoch 2/3, Batch 400/1982, Loss: 0.0026
  Epoch 2/3, Batch 600/1982, Loss: 0.0018
  Epoch 2/3, Batch 800/1982, Loss: 0.0018
  Epoch 2/3, Batch 1000/1982, Loss: 0.0015
  Epoch 2/3, Batch 1200/1982, Loss: 0.0015
  Epoch 2/3, Batch 1400/1982, Loss: 0.0020
  Epoch 2/3, Batch 1600/1982, Loss: 0.0015
  Epoch 2/3, Batch 1800/1982, Loss: 0.0017
Epoch 2/3 - Train Loss: 0.0019, Val Loss: 0.0018
  Epoch 3/3, Batch 200/1982, Loss: 0.0014
  Epoch 3/3, Batch 400/1982, Loss: 0.0016
  Ep

In [12]:
# Step 8: Make predictions and create submission (Memory-optimized)
print("=== Making Test Predictions & Creating Submission ===")

print(f"Test embeddings: {esm2_test_emb.shape}")
print(f"Test IDs: {len(esm2_test_ids)}")

top_k = 50  # Predict top 50 GO terms per protein (like baseline)
output_file = 'submission.tsv'

# Open file for writing
with open(output_file, 'w') as f:
        model.eval()
        batch_size = 32
        num_batches = (len(esm2_test_emb) + batch_size - 1) // batch_size
        total_predictions = 0
        
        with torch.no_grad():
            print(f"Starting prediction loop: {num_batches} batches total")
            for batch_idx in range(num_batches):
                start_idx = batch_idx * batch_size
                end_idx = min((batch_idx + 1) * batch_size, len(esm2_test_emb))
                
                # Get batch
                batch_emb = torch.tensor(esm2_test_emb[start_idx:end_idx], dtype=torch.float32).to(device)
                outputs = model(batch_emb)
                probs = torch.sigmoid(outputs).cpu().numpy()
                
                # Process each protein in batch and write directly to file
                for i, protein_idx in enumerate(range(start_idx, end_idx)):
                    protein_id = esm2_test_ids[protein_idx]
                    protein_probs = probs[i]
                    
                    # Get top-k GO terms (highest probabilities) - using argpartition for speed
                    top_k_indices = np.argpartition(protein_probs, -top_k)[-top_k:]
                    # Sort only the top-k (much faster than sorting all 25k)
                    top_k_indices = top_k_indices[np.argsort(protein_probs[top_k_indices])][::-1]
                    
                    # Write to file immediately
                    for go_idx in top_k_indices:
                        go_term = all_go_terms[go_idx]
                        confidence = float(protein_probs[go_idx])
                        f.write(f"{protein_id}\t{go_term}\t{confidence}\n")
                        total_predictions += 1
            
            # Free memory
            del batch_emb, outputs, probs
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            # Print progress more frequently
            if (batch_idx + 1) % 50 == 0:
                print(f"  Processed {batch_idx + 1}/{num_batches} batches, {total_predictions} predictions so far")
            if (batch_idx + 1) % 200 == 0:
                gc.collect()

print(f"\n✅ Submission created: {output_file}")
print(f"   Total predictions: {total_predictions}")
print(f"   File size: {os.path.getsize(output_file) / (1024*1024):.2f} MB")

=== Making Test Predictions & Creating Submission ===
Test embeddings: (141864, 1280)
Test IDs: 141864
Starting prediction loop: 4434 batches total

✅ Submission created: submission.tsv
   Total predictions: 7093200
   File size: 260.77 MB


In [13]:
# Step 9: Verify submission file
print("=== Verifying Submission ===")

# Count lines in file
with open(output_file, 'r') as f:
    line_count = sum(1 for _ in f)

print(f"Total lines in submission: {line_count}")

# Show first few lines
print("\nFirst 5 lines:")
with open(output_file, 'r') as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.strip())
        else:
            break

print(f"\n✅ Submission file ready: {output_file}")

=== Verifying Submission ===
Total lines in submission: 7093200

First 5 lines:
Q9ZSA8	GO:0005515	0.0990348532795906
Q9ZSA8	GO:0005634	0.08254110813140869
Q9ZSA8	GO:0005829	0.06589846312999725
Q9ZSA8	GO:0005739	0.04138588160276413
Q9ZSA8	GO:0005886	0.0381326787173748

✅ Submission file ready: submission.tsv
